# HTTP
In deze opdracht ga je zelf HTTP requests doen.
Je gaat de webpagina opvragen die het cijfer bevat van het aantal besmettingen van gisteren.

## Stap 1: de webpagina
We gaan een HTTP request doen bij de volgende webpagina: 
**https://coronadashboard.rijksoverheid.nl/landelijk/positief-geteste-mensen**

Neem ook eens een kijkje door de volgende regel op te zoeken in je browser:
```
view-source:https://coronadashboard.rijksoverheid.nl/landelijk/positief-geteste-mensen
```

Kun je het cijfer vinden van het aantal besmettingen van gisteren?

## Stap 2: HTTP GET request
We gaan een HTTP GET request doen. Hiervoor gebruiken we de **requests** library:

In [1]:
import requests

De URL (Uniform Resource Locator) van het HTTP GET request is:

In [2]:
url = 'https://coronadashboard.rijksoverheid.nl/landelijk/positief-geteste-mensen'

We gaan nu in Python een HTTP GET request doen:

In [3]:
r = requests.get(url)

## Stap 3: response bekijken

Als eerste gaan we de status code bekijken

In [4]:
print(r.status_code)

200


Is de status code 200? Dan is het gelukt!

Leg uit wat de volgende status codes betekenen:
- 404
- 414
- 418
- 429

Gelukkig bevat een response nog veel meer informatie:

In [5]:
for header, waarde in r.headers.items():
    print(header, waarde)

Content-Type text/html
Cache-Control no-cache, public
Content-Encoding gzip
Expires Mon, 15 Mar 2021 16:37:04 GMT
Last-Modified Mon, 15 Mar 2021 16:09:33 GMT
ETag W/"604f86bd-1471f8"
Content-Security-Policy default-src 'self' statistiek.rijksoverheid.nl; img-src 'self' statistiek.rijksoverheid.nl data:; style-src 'self' 'unsafe-inline'; script-src 'self' statistiek.rijksoverheid.nl;
Referrer-Policy origin
X-Content-Type-Options nosniff
X-Frame-Options SAMEORIGIN
X-XSS-Protection 1; mode=block
Date Tue, 16 Mar 2021 12:23:26 GMT
Content-Length 183170
Connection keep-alive
Vary Accept-Encoding
Strict-Transport-Security max-age=15768000 ; preload


Leg uit wat de volgende headers betekenen:
- Content-Type
- Last-Modified
- Date
- Content-Length

## Stap 4: op zoek naar het aantal besmettingen

We gaan eerst de HTML inlezen met Python via de lxml library.

In [6]:
from lxml import etree

In [7]:
doc = etree.HTML(r.text)

In [8]:
html_element = 'div'
attribuut_waarde = 'infected'
html_elementen = doc.xpath(f'//{html_element}[@data-cy="{attribuut_waarde}"]')

Als het goed is, is er maar een HTML element *div* met als waarde van **data-cy**: **infected**

In [9]:
len(html_elementen)

1

In [10]:
element = html_elementen[0]
etree.dump(element)

<div color="data.primary" data-cy="infected" class="kpi-value__StyledValue-sc-2mskn4-0 hZojKL">5.549</div>


In [11]:
aantal_besmettingen = element.text.replace('.', '')
print(aantal_besmettingen)

5549


## Stap 5: JSON
HTML is niet echt makkelijk in te lezen door computers. Het RIVM biedt ook ander soorten formaten aan:
https://data.rivm.nl/covid-19/

We gaan nu een HTTP request doen om een JSON bestand op te vragen voor het aantal besmettingen per gemeente per dag.

https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.json

In [12]:
page = 'https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.json'

In [13]:
r = requests.get(page)

In [14]:
print(r.status_code)

200


In [15]:
json_object = r.json()

In [16]:
stad = 'Amersfoort'
datum = '2020-04-10' # JAAR-MAAND-DAG
for informatie in json_object:
    if informatie['Municipality_name'] == stad:
        if informatie['Date_of_publication'] == datum:
            print(informatie)

{'Date_of_report': '2021-03-15 10:00:00', 'Date_of_publication': '2020-04-10', 'Municipality_code': 'GM0307', 'Municipality_name': 'Amersfoort', 'Province': 'Utrecht', 'Security_region_code': 'VR09', 'Security_region_name': 'Utrecht', 'Municipal_health_service': 'GGD Regio Utrecht', 'ROAZ_region': 'Traumazorgnetwerk Midden-Nederland', 'Total_reported': 2, 'Hospital_admission': 3, 'Deceased': 1}


* Hoeveel overledenen waren er in Amersfort op 27 november 2020? En hoeveel ziekenhuisopnames?
* Hoeveel positieve besmettingen had Urk op oudjaarsdag 2020?